※ このノートブックで扱うモデルの言語は英語となります。

# 教育のための大規模言語モデル

このノートブックでは、大規模言語モデル(LLM)を教育のユースケースに使う方法を示します。 
LLMは要約、質問応答、質問と回答のペアの生成などのタスクに使用できます。

このノートブックの最初の部分では、単一の `ml.p3.2xlarge` インスタンス上で **FLAN T5** モデルを使用して、SageMaker エンドポイントをセットアップします。 次に、エンドポイントへどのようにクエリできるかを示し、要約、質問応答、質問と回答のペアの生成のプロンプトを示すためにいくつかの小さなプログラムを追加します。

最後のセクションは、量子コンピューティングに関する Wikipedia の記事、[Project Gutenberg](https://www.gutenberg.org/)の ebook テキスト、arxiv.org の科学的的な記事(pdf)、オーストラリアの 2023-24 年の予算に関する Medicare Overview のクエリを扱う、4 つのデモに分かれています。

## 1.SageMaker Endpoint の設定

### 1.1 Python 依存パッケージのインストール

In [24]:
%%capture
!pip install --upgrade pip
!pip install -U sagemaker
!pip install -U langchain
!pip install -U PyPDF2

### 1.2 SageMaker Endpoint のデプロイ

In [25]:
import sagemaker, boto3, json, logging
from sagemaker import image_uris, instance_types, model_uris, script_uris
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.session import Session
from sagemaker.utils import name_from_base
from IPython.display import display, HTML, IFrame

In [26]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [27]:
logger.info(f'Using sagemaker=={sagemaker.__version__}')
logger.info(f'Using boto3=={boto3.__version__}')

Using sagemaker==2.145.0
Using boto3==1.26.111


In [ ]:
# モデルの重みを保存するフォルダの作成
!mkdir -p download_dir
!mkdir -p source_documents_dir

In [29]:
def get_sagemaker_session(local_download_dir) -> sagemaker.Session:
    """Return the SageMaker session."""

    sagemaker_client = boto3.client(
        service_name="sagemaker", region_name=boto3.Session().region_name
    )

    session_settings = sagemaker.session_settings.SessionSettings(
        local_download_dir=local_download_dir
    )

    session = sagemaker.session.Session(
        sagemaker_client=sagemaker_client, settings=session_settings
    )

    return session

In [30]:
sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

model_id, model_version = "huggingface-text2text-flan-t5-xl", "*"
_model_env_variable_map = {
    "huggingface-text2text-flan-t5-xl": {"MMS_DEFAULT_WORKERS_PER_MODEL": "1"},
}

endpoint_name = name_from_base(f"jumpstart-example-{model_id}")
instance_type = 'ml.p3.2xlarge'
logger.info(f'Using role {aws_role} in region {aws_region}')

Using role arn:aws:iam::278313627171:role/service-role/AmazonSageMaker-ExecutionRole-20221130T152651 in region us-east-1


In [ ]:
# HuggingFace のコンテナイメージをベースにした、推論用 docker イメージの URI を取得します。
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # model_idから自動的に推定されます。
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=instance_type,
)

# モデルの読み込みや推論リクエストを処理するためのスクリプトの URI を取得します。このスクリプトには依存パッケージが含まれます。
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)

# モデルの URI を取得します。
model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)

# SageMaker Model を作成します。
if model_id in _model_env_variable_map:
    # 大きなモデルに対して、私たちは既に推論スクリプトとモデルの詰め合わせを提供しています。
    # そのため、（スクリプトのアップロードに使用される）source_dir は必須ではありません。
    model = Model(
        image_uri=deploy_image_uri,
        model_data=model_uri,
        role=aws_role,
        predictor_cls=Predictor,
        name=endpoint_name,
        env=_model_env_variable_map[model_id],
    )
else:
    model = Model(
        image_uri=deploy_image_uri,
        source_dir=deploy_source_uri,
        model_data=model_uri,
        entry_point="inference.py",  # deploy_source_uri に含まれる entry point となるファイル
        role=aws_role,
        predictor_cls=Predictor,
        name=endpoint_name,
        sagemaker_session=get_sagemaker_session("download_dir"),
    )

In [32]:
# # The following requires "servicequotas:GetServiceQuota"
# sq = boto3.client('service-quotas')
# response = sq.get_service_quota(
#     ServiceCode='sagemaker',
#     QuotaCode='L-1623D0BE'
# )
# quota = response['Quota']
# print(f"{quota['QuotaName']}: {quota['Value']}")  # ml.p3.2xlarge for endpoint usage: 2.0

In [33]:
%%time
# モデルをデプロイします。Model クラスを使ってデプロイする場合、SageMaker API で推論するため、
# Predictor クラスを受け取る必要があります。
print(f'Deploying endpoint {endpoint_name} on 1 x {instance_type} (this will take approximately 6-8 minutes)')
try:
    model_predictor = model.deploy(
        initial_instance_count=1,
        instance_type=instance_type,
        predictor_cls=Predictor,
        endpoint_name=endpoint_name,
    )
except Exception as e:
    print(f'Error: {e}')
    print('Two common reasons for this error')
    print('1. You are in a AWS region that does not have the ml.p3.2xlarge instance type')
    print('2. You have exceeded the service quota of this AWS account')

Creating model with name: jumpstart-example-huggingface-text2text-2023-06-10-08-37-25-049
CreateModel request: {
    "ModelName": "jumpstart-example-huggingface-text2text-2023-06-10-08-37-25-049",
    "ExecutionRoleArn": "arn:aws:iam::278313627171:role/service-role/AmazonSageMaker-ExecutionRole-20221130T152651",
    "PrimaryContainer": {
        "Image": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04",
        "Environment": {
            "MMS_DEFAULT_WORKERS_PER_MODEL": "1"
        },
        "ModelDataUrl": "s3://jumpstart-cache-prod-us-east-1/huggingface-infer/prepack/v1.1.0/infer-prepack-huggingface-text2text-flan-t5-xl.tar.gz"
    },
    "Tags": [
        {
            "Key": "aws-jumpstart-inference-model-uri",
            "Value": "s3://jumpstart-cache-prod-us-east-1/huggingface-infer/prepack/v1.1.0/infer-prepack-huggingface-text2text-flan-t5-xl.tar.gz"
        }
    ]
}


Deploying endpoint jumpstart-example-huggingface-text2text-2023-06-10-08-37-25-049 on 1 x ml.p3.2xlarge (this will take approximately 6-8 minutes)


Creating endpoint-config with name jumpstart-example-huggingface-text2text-2023-06-10-08-37-25-049
Creating endpoint with name jumpstart-example-huggingface-text2text-2023-06-10-08-37-25-049


------------!CPU times: user 137 ms, sys: 14.1 ms, total: 151 ms
Wall time: 6min 33s


In [34]:
print(f'Successfully deployed endpoint {endpoint_name} on 1 x {instance_type}')

Successfully deployed endpoint jumpstart-example-huggingface-text2text-2023-06-10-08-37-25-049 on 1 x ml.p3.2xlarge


## 2. テキスト抽出とエンドポイントへクエリするためのサブルーチンの作成

In [ ]:
import pandas as pd
import PyPDF2
import re
import requests

from bs4 import BeautifulSoup

pd.set_option('max_colwidth', 80)  # Pandas Dataframe を表示するための最大幅を指定します。
QNA_OUTPUT_STYLE = 'HTML'

### 2.1 PDF ファイルからのテキスト抽出

PyPDF ライブラリを使用して、PDFファイルからテキストを抽出できます。（英語）

In [36]:
def extract_pages(pdf_file, max_pages=100):
    pages = []
    with open(pdf_file, 'rb') as f:
        for i, page in enumerate(PyPDF2.PdfReader(f).pages):
            if i == max_pages:
                break
            pages.append(page.extract_text())
    return pages

### 2.2 URL からテキストベースのebookをダウンロード

In [37]:
def download_url_text(url):
    r = requests.get(url)
    if r.status_code == 200:
        return r.content.decode('utf-8')
    else:
        print(f'Failed to download {url}. Status code = {r.status_code}')
        return None

### 2.3 ウェブページから項目を抽出

Beatiful Soup ライブラリを使用して、ウェブページのHTMLコンテンツをパースできます。
ここでは、`<body>` セクションの中の `<p>` タグを抽出します。 

In [38]:
def extract_paragraphs_from_html(text):
    html = BeautifulSoup(text, 'html.parser')
    return [ p.text for p in html.body.select('p') ]

### 2.4 Generate texts, or query an endpoint with prompts

### 2.4 テキストの生成とプロンプトを使ったエンドポイントへのクエリ

In [39]:
newline, bold, unbold = '\n', '\033[1m', '\033[0m'
lightred, lightgreen, lightyellow, lightblue = '\033[91m', '\033[92m', '\033[93m', '\033[94m'
lightmagenta, lightcyan, reset = '\033[95m', '\033[96m', '\33[39m'

def query_endpoint_with_json_payload(encoded_json):
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=encoded_json)
    return response

def parse_response_multiple_texts(query_response):
    model_predictions = json.loads(query_response['Body'].read())
    generated_text = model_predictions['generated_texts']
    return generated_text

def generate_text_from_prompt(prompt, max_length=300, max_time=50, temperature=0.5,
                              top_k=None, top_p=None, do_sample=True, seed=None):
    payload = {
        "text_inputs": prompt,
        "max_length": max_length,
        "max_time": max_time,
        "temperature": temperature,
        "do_sample": do_sample
    }
    if top_k is not None:
        payload['top_k'] = top_k
    if top_p is not None:
        payload['top_p'] = top_p
    if seed is not None:
        payload['seed'] = seed

    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    return parse_response_multiple_texts(query_response)[0]

大規模言語モデルの出力をさらにカスタマイズするために、以下のパラメータが利用可能です：

* **max_length:**　モデルは出力長（入力コンテキストの長さを含む）が`max_length`に達するまでテキストを生成します。指定する場合は正の整数である必要があります。
* **max_time:** 計算の実行を許可する最大時間を秒単位で指定します。この設定は、エンドポイント呼び出しの応答タイムアウトエラーが発生する前に応答を生成するのに役立ちます。
* **num_return_sequences:** 返される出力列の数。指定する場合は正の整数である必要があります。
* **num_beams:** 貪欲探索に使用されるビームの数。`num_return_sequences` 以上の整数である必要があります。

* **no_repeat_ngram_size:** モデルが`no_repeat_ngram_size`の単語列が出力列で繰り返されないことを保証します。指定する場合は、1以上の正の整数である必要があります。
* **temperature:** 出力のランダム性を制御します。temperature が高いほど確率の低い単語が出力され、低いほど確率の高い単語が出力されます。もし `temperature` -> 0 ならば、貪欲なデコード結果（確率が最大となる単語の列）となります。指定する場合は正の浮動小数点数である必要があります。
* early_stopping:** Trueの場合、全ての仮説が文末トークンに到達した時点でテキスト生成を終了します。指定する場合、booleanである必要があります。
* **do_sample:** Trueの場合、サンプリング戦略が有効になります。指定する場合、booleanである必要があります。
* **top_k:** テキスト生成の各ステップにおいて、最も可能性の高い単語 `top_k` のみをサンプリングします。指定する場合、正の整数である必要があります。
* **top_p:** テキスト生成の各ステップにおいて、累積確率 `top_p` で可能な最小の単語セットからサンプリングします。指定する場合、0から1の間の浮動小数点である必要があります。
* **seed:** 再現性のためにランダム状態を固定します。指定する場合は整数である必要があります。


In [40]:
def summarize(text, seed=None):
    return generate_text_from_prompt(
        f"""Summarize the following text in 100 words:\n\n{text}\n\nSummary:""",
        temperature=0.2,  # 要約のための低い temperature
        seed=seed
    )

def ask(context, question, seed=None):
    return generate_text_from_prompt(
        f"""CONTEXT:\n{context}\n{question}""",
        temperature=0.01,  # 正確性のための最も低いtempereature
        max_length=150,    # 回答が冗長すぎないように設定
        seed=seed
    )

def extract_question(text, seed=None):
    return generate_text_from_prompt(
        f"""EXTRACT QUESTIONS\nContext:\n{text}\nQuestion:""",
        temperature=1.0,  # 創造性のための最大のtemperature
        seed=seed
    )

def create_qna_pairs(text, n, output_style='HTML', seed=None):
    questions = []
    answers = []

    for i in range(n):
        qn = extract_question(text, seed) if i == 0 else extract_question(text)
        questions.append(qn)
        answers.append(ask(text, qn))
        if output_style == 'HTML':
            output = \
            f"""<b>{i+1}</b>. <b><font color=#FF7F50>Question</font></b>: {questions[i]}
            <b><font color=#FA8072>Answer</font></b>: {answers[i]}"""
            display(HTML(output))
        elif output_style == 'text':
            print(f"""{i+1}. {lightblue}{bold}Question{unbold}{reset}: {questions[i]} {lightcyan}{bold}Answer{unbold}{reset}: {answers[i]}""")
    if output_style == 'table':
        return pd.DataFrame({
            'Question': questions,
            'Answer': answers
        }).drop_duplicates()

## 3. LLM Demos for Education

## 3. 教育のための LLM デモ

このノートブックでは、以下のテキストを使ってテキストの要約と、質問と回答のペアの生成を行います。

1. Wikipediaの量子コンピューティングに関する記述: https://en.wikipedia.org/wiki/Quantum_computing
<!-- 1. Quantum Computing and Quantum Information (by Nielsen & Chuang): https://michaelnielsen.org/qcqi/QINFO-book-nielsen-and-chuang-toc-and-chapter1-nov00.pdf (this is a sample chapter from [this website](https://michaelnielsen.org/qcqi/)) -->
2. くまのプーさん (by Alan Alexander Milne): https://www.gutenberg.org/ebooks/67098.txt.utf-8
3. アテンションに関する論文 Attention is all you need (by Vaswani et al): https://arxiv.org/pdf/1706.03762.pdf
4. オーストラリアの2023-24年の予算概要: https://budget.gov.au/content/overview/download/budget_overview-20230511.pdf

このノートブックでは、シンプルで簡単にデプロイできる Flan T5 XL モデルを使っています。
さらに良い結果を得るためにファインチューニングや改善されたモデルを使うことができます。

In [ ]:
# curl コマンドを利用して、pdfとテキストをダウンロードしあmす。
# リダイレクトを許可する `-L` とサイレントモードの `-s`、出力ファイルを指定するための `-o` を指定しています。

# アテンションに関する論文 Attention is all you need (by Vaswani et al)
!curl -Ls https://arxiv.org/pdf/1706.03762.pdf -o source_documents_dir/attention.pdf
# オーストラリアの2023-24年の予算概要
!curl -Ls https://budget.gov.au/content/overview/download/budget_overview-20230511.pdf -o source_documents_dir/aus_budget_overview-2023-24.pdf

### 3.1 量子コンピュータに関するWikipediaのページ

この例では、量子コンピュータに関する Wikipedia のページを context として使用します。
LLM は キーワード生成、要約、質問と回答のペア作成 に使います。
好みに合わせて、WikipediaのURL、ウェブサイトやブログ、新しい記事になどに変更できます。

In [ ]:
NCHARS = 400     # 抽出したテキストの最初の400文字を使います。より多くのコンテキストが必要なら増やします。
NQUESTIONS = 10  # 生成する質問回答のペアの個数

In [ ]:
wiki_paragraphs = extract_paragraphs_from_html(
    download_url_text('https://en.wikipedia.org/wiki/Quantum_computing')
)[1:11]  # 最初の2つのパラグラフは飛ばします。
wiki_txt = '\n\n'.join(wiki_paragraphs)
# print(f'{txt1[:NCHARS]}...\n\n...{txt1[-NCHARS:]}')
IFrame('https://en.wikipedia.org/wiki/Quantum_computing', width=800, height=300)

#### Key word Generation

#### キーワード生成

In [44]:
KEY_WORDS = generate_text_from_prompt(
    f'FIND KEY WORDS\n\nContext:\n{wiki_txt}\nKey Words:',
    seed=12345
)
key_word_list = KEY_WORDS.split(', ')
print(KEY_WORDS)

quantum, computing, superposition, qubit


#### キーポイントの要約

パラグラフごとに、短い要約を作成します。

In [45]:
summary = []
for i, x in enumerate(wiki_paragraphs):
    summary.append(f'{i+1}. {summarize(x[:1500])}')

In [46]:
HTML(
    '<h4>Key Points</h4>' + 
    '\n'.join([ f'<li>{x}</li>' for x in summary ])
)

In [ ]:
# 上記の 10 個のポイントは短い要約をつくるために使うことができます。
summarize('\n'.join(summary))

#### 正答であるかのチェック

この例では、テキストをベースに "正しい回答" を作成します。
生徒が作成した、誤った回答 と 正しい回答（公式の回答は言い方が異なる正答） を1つずつ作成したとします。
LLMを使って生徒の回答が正しいかどうかをチェックします。

In [48]:
prompt=f"""Context:{wiki_txt}
What is quantum computing?"""
answer = generate_text_from_prompt(prompt, temperature=0.01)
print(answer)

A quantum computer is a computer that exploits quantum mechanical phenomena.


In [49]:
prompt=f"""Context:{wiki_txt}
Question: What is quantum computing?
Answer: {answer}
Student: Quantum computing is using computers with quantum dots
Is this answer correct?"""
print(generate_text_from_prompt(prompt, temperature=0.01))

no


In [50]:
prompt=f"""Context:{wiki_txt}
Question: What is quantum computing?
Answer: {answer}
Student: Quantum computing involves using computers that make use of quantum mechanics
Is this answer correct?"""
print(generate_text_from_prompt(prompt, temperature=0.01))

yes


#### 質問と回答のペア生成

In [51]:
create_qna_pairs(wiki_txt, NQUESTIONS, output_style=QNA_OUTPUT_STYLE, seed=1234)

### 3.2 くまのプーさん (by Alan Alexander Milne)

In [ ]:
winnie_the_pooh = download_url_text('https://www.gutenberg.org/ebooks/67098.txt.utf-8')

In [53]:
x = winnie_the_pooh.find('CHAPTER III')
pooh_txt = winnie_the_pooh[x:x+5000]  # Extract the first 5000 characters of chapter 3
print(pooh_txt)

CHAPTER III

                   IN WHICH POOH AND PIGLET GO HUNTING
                        AND NEARLY CATCH A WOOZLE


The Piglet lived in a very grand house in the middle of a beech-tree,
and the beech-tree was in the middle of the forest, and the Piglet lived
in the middle of the house. Next to his house was a piece of broken
board which had: "TRESPASSERS W" on it. When Christopher Robin asked the
Piglet what it meant, he said it was his grandfather's name, and had
been in the family for a long time, Christopher Robin said you
_couldn't_ be called Trespassers W, and Piglet said yes, you could,
because his grandfather was, and it was short for Trespassers Will,
which was short for Trespassers William. And his grandfather had had two
names in case he lost one--Trespassers after an uncle, and William after
Trespassers.

"I've got two names," said Christopher Robin carelessly.

"Well, there you are, that proves it," said Piglet.

One fine winter's day when Piglet was brushing away the s

In [ ]:
x = winnie_the_pooh.find('CHAPTER III')
pooh_txt = winnie_the_pooh[x:x+5000]  # チャプター3の最初の5000文字を抽出します
print(pooh_txt)

In [54]:
ask(pooh_txt, "What is the storyline here?")

'The Piglet lived in a very grand house in the middle of a beech-tree, and the beech-tree was in the middle of the forest, and the Piglet lived in the middle of the house. Next to his house was a piece of broken board which had: "TRESPASSERS W" on it. When Christopher Robin asked the Piglet what it meant, he said it was his grandfather\'s name, and had been in the family for a long time. Christopher Robin said you _could_ be called Trespassers W, and Piglet said yes, you could, because his grandfather was, and it was short for Trespassers Will'

In [55]:
ask(pooh_txt, "Who is the main character?")

'Winnie-the-Pooh'

In [56]:
ask(pooh_txt, "What happens at the end?")

'Winnie-the-Pooh and Piglet nearly catch a Woozle.'

In [57]:
create_qna_pairs(pooh_txt, NQUESTIONS, output_style=QNA_OUTPUT_STYLE, seed=12345)

### 3.3 アテンションに関するAttention is all you need (by Vaswani et al)

In [58]:
attention = extract_pages('source_documents_dir/attention.pdf')

In [59]:
attention_txt = '\n\n'.join(attention[1:3] + attention[9:10])  # We will use pages 1, 2 (for the intro), and 9 (for the conclusion)
# print(f'{attention_txt[:NCHARS]}...\n\n\n...{attention_txt[-NCHARS:]}')
IFrame('source_documents_dir/attention.pdf', width=800, height=400)

#### 質問回答

In [60]:
ask(attention_txt, "What is the main gist of the paper?")

'We propose the Transformer, a model architecture eschewing recurrence and instead relying entirely on an attention mechanism to draw global dependencies between input and output. The Transformer allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs.'

In [61]:
ask(attention_txt, "What is the problem being solved?")

'The Transformer is the first sequence transduction model relying entirely on attention, replacing the recurrent layers most commonly used in encoder-decoder architectures with multi-headed self-attention.'

In [62]:
ask(attention_txt, "What is the conclusion of the paper?")

'Transformer is the first sequence transduction model based entirely on attention, replacing the recurrent layers most commonly used in encoder-decoder architectures with multi-headed self-attention.'

In [63]:
chunk_size = len(attention_txt)//8
print(f'The text will be split up into chunks of {chunk_size} characters and summarized')

The text will be split up into chunks of 1202 characters and summarized


In [64]:
display(HTML('<h4>Key Points</h4>'))
summary = []
for i in range(8):
    x0 = i*chunk_size
    x1 = (i+1)*chunk_size
    line_summary = f'{i+1}. {summarize(attention_txt[x0:x1])}'
    display(HTML(line_summary))
    summary.append(line_summary)

In [65]:
create_qna_pairs(attention_txt, NQUESTIONS, output_style=QNA_OUTPUT_STYLE)

### 3.4 オーストラリアの 2023-24年の予算概要 (Medicare)

この例では、オーストラリアの 2023-24年の予算をみて、メディケアの改善に着目します。

In [66]:
# Extracting the pages from the Budget overview and work on the pages 24 to 27 (Medicare related)
aus_budget_overview = extract_pages('source_documents_dir/aus_budget_overview-2023-24.pdf')
txt_aus_budget_overview_medicare = '\n\n'.join(aus_budget_overview[24:27])  # We will use pages 24 to 27. Those pages cover the Medicare budget.
print(f'{txt_aus_budget_overview_medicare[:NCHARS]}...\n\n\n...{txt_aus_budget_overview_medicare[-NCHARS:]}')

Historic investment in Medicare 
Strengthening Medicare
Medicare is the foundation of Australia’s primary health care system. In this 
Budget, the Government is investing $5.7 billion over 5 years from 2022—23 to 
strengthen Medicare and make it cheaper and easier to see a doctor.
The Strengthening Medicare package includes the largest investment in bulk 
billing incentives ever. The Government is...


...llion over 4 years to establish the Primary Care and Midwifery 
Scholarships program, supporting registered nurses and midwives in 
post-graduate study to improve their skills 
• $31.6 million over 2 years for improved training arrangements for 
international medical students working rur al and remote locations.
26 Strengthening MedicareStronger foundations for a better future   |   Budget 2023–24


In [ ]:
# メディケアに関連する 24 から 27 ページを抽出します。
aus_budget_overview = extract_pages('source_documents_dir/aus_budget_overview-2023-24.pdf')
txt_aus_budget_overview_medicare = '\n\n'.join(aus_budget_overview[24:27])  # メディケアの予算に関する予算をカバーする 24 から 27 ページを使います。
print(f'{txt_aus_budget_overview_medicare[:NCHARS]}...\n\n\n...{txt_aus_budget_overview_medicare[-NCHARS:]}')

#### 質問回答

In [67]:
summarize(txt_aus_budget_overview_medicare)

'The Government is investing $5.7 billion over 5 years from 2022-23 to strengthen Medicare and make it cheaper and easier to see a doctor.'

In [68]:
create_qna_pairs(txt_aus_budget_overview_medicare, 5, output_style=QNA_OUTPUT_STYLE)

In [69]:
ask(txt_aus_budget_overview_medicare,"What is a Level B consultation?")

'telehealth general practice services which are between 6 and 20 minutes in length'

In [70]:
ask(txt_aus_budget_overview_medicare, "How much is the govermement investing?")

'$5.7 billion'

In [71]:
ask(txt_aus_budget_overview_medicare, "Is the governement helping the homeless people?")

'The Government will also invest in new services to help homeless people and culturally and linguistically diverse communities to access primary care.'

## 4. [Optional] Education Part II デモ

このセクションでは、URL を受け取る Gradio アプリケーションを作成し、ウェブページのコンテンツをベースに質問回答を行います。

### 4.1 Gradio Demo App

In [72]:
%%capture
!pip install gradio

In [73]:
import gradio as gr

In [74]:
def url2context(url):
    paragraph_list = extract_paragraphs_from_html(
        download_url_text(url)
    )[1:11]  # We will skip the first paragraph, and take only 10 paragraphs
    return '\n\n'.join(paragraph_list)

In [75]:
def chatbot(prompt, temperature, max_length, url):
    if url == "":
        return generate_text_from_prompt(prompt, max_length, temperature)
    else:
        return ask(url2context(url), prompt)

def summary(url):
    context = url2context(url)
    key_words = generate_text_from_prompt(
        f'FIND KEY WORDS\n\nContext:\n{context}\nKey Words:'
    )
    return f"""{summarize(context)}\n\nKey words: {key_words}"""

with gr.Blocks() as demo:
    gr.Markdown("## Flan T5 Chatbot Demo")
    with gr.Row():
        with gr.Column():
            url = gr.Textbox(label="URL", placeholder="Enter URL here", lines=1, show_label=True,
                             value="https://mmrjournal.biomedcentral.com/articles/10.1186/s40779-022-00416-w"
                             # value="https://k12.libretexts.org/Bookshelves/Science_and_Technology/Biology/03%3A_Genetics/3.14%3A_Human_Genome"
                            )
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(
                label="Prompt", placeholder="Enter your prompt here", lines=3, show_label=True,
                value=f"How do mRNA vaccines work for pancreatic cancer treatment?")
            temperature = gr.Slider(label="Temperature", minimum=0.0, maximum=1.0, value=0.5)
            max_length = gr.Slider(label="Max Length", minimum=20, maximum=400, value=100)
        with gr.Column():
            output = gr.Textbox(label="Output", lines=10, show_label=True)
    with gr.Row():
        with gr.Column():
            submit_btn = gr.Button("Submit")
        with gr.Column():
            summary_btn = gr.Button("Summary")
    submit_btn.click(
        fn=chatbot,
        inputs=[prompt, temperature, max_length, url],
        outputs=output,
        api_name="chatbot",
        queue=False
    )
    summary_btn.click(
        fn=summary,
        inputs=[url],
        outputs=output,
        api_name="summary",
        queue=False
    )

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://c60adc167cc2915343.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## 5. クリーンアップ

SageMaker モデルとエンドポイントを削除します。

In [76]:
model_predictor.delete_model()
model_predictor.delete_endpoint()

Deleting model with name: jumpstart-example-huggingface-text2text-2023-06-10-08-37-25-049
Deleting endpoint configuration with name: jumpstart-example-huggingface-text2text-2023-06-10-08-37-25-049
Deleting endpoint with name: jumpstart-example-huggingface-text2text-2023-06-10-08-37-25-049


File > Shut Down > Shutdown All と操作し、SageMakerを完全にシャットダウンします。